# Working with APIs

**Project Goal**: Obatining data with APIs and transforming it to a manageable format.

Specifics:

1. Extract stock data from the AlphaVantage API using a URL.

2. Extract stock data from AlphaVantage API using an HTTP request.

3.  Write a function for transforming stock data.

4.  Incorporate Python Exceptions into the function.

In [1]:
import pandas as pd 
import requests

### Accessing APIs Through a URL

In [2]:
url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo"

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo'

In [7]:
pd.read_json(url)["Time Series (Daily)"]

1. Information                                                     NaN
2. Symbol                                                          NaN
3. Last Refreshed                                                  NaN
4. Output Size                                                     NaN
5. Time Zone                                                       NaN
                                           ...                        
2023-12-05           {'1. open': '160.7600', '2. high': '162.4700',...
2023-12-04           {'1. open': '160.2900', '2. high': '162.7900',...
2023-12-01           {'1. open': '158.4100', '2. high': '160.5900',...
2023-11-30           {'1. open': '156.9500', '2. high': '158.6000',...
2023-11-29           {'1. open': '156.1500', '2. high': '157.5100',...
Name: Time Series (Daily), Length: 105, dtype: object